In [2]:
import os
import openai

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "")
openai.api_key = OPENAI_API_KEY

In [26]:
messages = [{"role": "system", "content": "Hello World"}]
response = openai.ChatCompletion.create(
    model="gpt-4o-mini-2024-07-18",
    messages=messages,
    n=1,
    stop=None,
)
print(response)
response.choices[0].message.content

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "Hello! How can I assist you today?",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1734502676,
  "id": "chatcmpl-AfhfYvk6IGBxI7vG5VauRLFrXjMpI",
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "system_fingerprint": "fp_6fc10e10eb",
  "usage": {
    "completion_tokens": 9,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 0,
      "rejected_prediction_tokens": 0
    },
    "prompt_tokens": 9,
    "prompt_tokens_details": {
      "audio_tokens": 0,
      "cached_tokens": 0
    },
    "total_tokens": 18
  }
}


'Hello! How can I assist you today?'

In [27]:
import json

SANDBOX_DIR = "/Users/aidand/dev/babyagi/sandbox"
if not os.path.exists(SANDBOX_DIR):
    os.makedirs(SANDBOX_DIR)

def create_file(path, content):
    with open(os.path.join(SANDBOX_DIR, path), "w") as f:
        f.write(content)
    print(f"Created file {os.path.join(SANDBOX_DIR, path)}")

tools = [
    {
        "type": "function",
        "function": {
            "name": "create_file",
            "description": "Create a file with the given name and content",
            "parameters": {
                "type": "object",
                "properties": {
                    "path": {
                        "type": "string",
                        "description": "The relative path to the file to create",
                    },
                    "content": {
                        "type": "string",
                        "description": "The content of the file to create",
                    },
                },
                "required": ["path", "content"],
            },
        },
    }
]

def maybe_create_file(prompt):  
    messages = [{"role": "system", "content": prompt}]
    response = openai.ChatCompletion.create(
        model="gpt-4o-mini-2024-07-18",
        messages=messages,
        n=1,
        stop=None,
        tools=tools,
    )
    print(response)
    message = response.choices[0].message

    if message.tool_calls and message.tool_calls[0].function.name == "create_file":
        tool_call = message.tool_calls[0]
        print("Generating tool call: ", tool_call)
        arguments = json.loads(tool_call.function.arguments)
        create_file(arguments["path"], arguments["content"])
    else:
        print(message)

maybe_create_file("Create me a file called 'hello.txt' with the content 'Hello World'")

{
  "choices": [
    {
      "finish_reason": "tool_calls",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": null,
        "refusal": null,
        "role": "assistant",
        "tool_calls": [
          {
            "function": {
              "arguments": "{\"path\":\"hello.txt\",\"content\":\"Hello World\"}",
              "name": "create_file"
            },
            "id": "call_aknnXFhTDkvGxDNncS8NIql1",
            "type": "function"
          }
        ]
      }
    }
  ],
  "created": 1734502697,
  "id": "chatcmpl-AfhftIbO1QTJsHNLjL03GUrpfyavX",
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "system_fingerprint": "fp_6fc10e10eb",
  "usage": {
    "completion_tokens": 20,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 0,
      "rejected_prediction_tokens": 0
    },
    "prompt_tokens": 81,
    "prompt_tokens_details": {
      "audio_tokens": 0,
 

In [24]:
maybe_create_file("Write a python script that prints 'Hello World'")

Generating tool call:  {
  "function": {
    "arguments": "{\"path\":\"hello_world.py\",\"content\":\"print('Hello World')\"}",
    "name": "create_file"
  },
  "id": "call_AxCPlnlPCSSAwzp6D8X3Z4Ff",
  "type": "function"
}
Created file /Users/aidand/dev/babyagi/sandbox/hello_world.py


In [25]:
maybe_create_file("Write me a python script that makes a call to the openai api to generate a response to the prompt 'Hello World'")

Generating tool call:  {
  "function": {
    "arguments": "{\"path\":\"openai_api_call.py\",\"content\":\"import openai\\nimport os\\n\\ndef call_openai_api(prompt):\\n    openai.api_key = os.getenv('OPENAI_API_KEY')\\n    response = openai.ChatCompletion.create(\\n        model='gpt-3.5-turbo',\\n        messages=[{'role': 'user', 'content': prompt}]\\n    )\\n    return response['choices'][0]['message']['content']\\n\\nif __name__ == '__main__':\\n    prompt = 'Hello World'\\n    response = call_openai_api(prompt)\\n    print(response)\"}",
    "name": "create_file"
  },
  "id": "call_bYpRikcwfNcOKgtJhPBUR2iR",
  "type": "function"
}
Created file /Users/aidand/dev/babyagi/sandbox/openai_api_call.py
